# Parse an IPAC table to csv and json for d3

*I will want to clean this up later when I decide the exact format for the interactive*

In [1]:
from astropy.table import Table
import numpy as np
import csv
import pandas as pd

from astropy import units as u
from matplotlib import pyplot as plt
from astropy.stats import LombScargle
from scipy.signal import argrelextrema

In [2]:
filename = "27882110006813.ipac_tbl"
ipac_lc = Table.read(filename, format='ipac')

hjd = np.array(ipac_lc["obsmjd"])
mag = np.array(ipac_lc["mag_autocorr"])
mag_unc = np.array(ipac_lc["magerr_auto"])

mmag = np.mean(mag) - mag

*Write to the file*

In [3]:
csvfile = open("27882110006813.csv", 'wt')
csvwriter = csv.writer(csvfile)
csvwriter.writerow(["hjd","mag","emag"])
for i in range(len(hjd)):
    csvwriter.writerow([hjd[i], mmag[i], mag_unc[i]])
csvfile.close()

### Create the periodogram output file

In [4]:
# Note: these are all in units of days
############
# periodogram and best-fit period
pmin = 0.1
pmax = 50.
ls = LombScargle(hjd, mag)
frequency, power = ls.autopower(maximum_frequency=1./pmin, minimum_frequency=1./pmax)
best_frequency = frequency[np.argmax(power)]
period = 1./frequency
best_period = 1./best_frequency
print("best period", best_period)

############
# Get the harmonics f/2, f/3
htests = np.array([best_frequency, best_frequency/2., best_frequency/3.])

############
# Get a few common failures : frequency +/- integers (for a typical 1 day observing cadence) 
ftests = np.array([])
rng = np.array([1, 2])
for i in rng:
    hm = [htests + i, htests - i]
    ftests = np.append(ftests, hm)
ftests = np.unique(ftests) #also sorts
useit = np.where(ftests > 0)
ftests = ftests[useit]

############
#Simply get some of the top maxima, but exclude any that have a high false-alarm probability
spacing = 50 # number of neighboring points to consider for finding relative maxima
alarm = 1e-4 # maximum of false alarm probability to accept
posall = argrelextrema(power, np.greater, order=spacing)[0] #get all maxima
fx = frequency[posall]
powx = power[posall]

# test the false-alarm probability
falarm = ls.false_alarm_probability(powx)
useit = np.where(falarm < alarm)
ftests2 = fx[useit]


############
#Sort these in order of importance for the user.  
#I will keep the best fit and harmonics first, then append a combined list of these last two sorted by power
#Combine the common failures and extra maxima
ftAll = np.append(ftests, ftests2)
ptAll = np.interp(ftAll, frequency, power)
ft, fi = np.unique(ftAll, return_index=True)
pt = ptAll[fi]
sp = np.argsort(pt)
sp = sp[::-1]
ft = ft[sp]
# remove the harmonics from this list
ft1 = np.array([])
for f in ft:
    test = np.where(htests == f)[0]
    if (len(test) == 0):
        ft1 = np.append(ft1, f)
        
# now prepend the harmonics 
ftest_final = np.append(htests, ft1)
ptest_final = 1/ftest_final

print(ptest_final)

best period 0.4661477095501411
[ 0.46614771  0.93229542  1.39844313  0.87527406  0.3176634   0.87317731
  7.01791922  0.31794048  1.16333985  0.24108044  0.53752377  6.88502467
  0.24124039  0.58306287 13.77004933  0.36831315  0.48248079  0.325455  ]


*Write to the file*

In [5]:
csvfile = open("27882110006813_periods.csv", 'wt')
csvwriter = csv.writer(csvfile)
csvwriter.writerow(["period"])
for i in range(len(ptest_final)):
    csvwriter.writerow([ptest_final[i]])
csvfile.close()

### Write everything to a single json file

In [7]:
outDict = {}
outDict['obsmjd'] = hjd - np.min(hjd)
outDict['mag_autocorr_mean'] = mmag
outDict['magerr_auto'] = mag_unc
outDict['period'] = best_period
pd.Series(outDict).to_json("27882110006813.json", orient='index')